In [ ]:
#import libraries
import rasterio as rio
from rasterio.windows import Window
from matplotlib import pyplot as plt
import numpy as np
from itertools import product
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import os
from osgeo import gdal

In [ ]:
#read standard spectra (available in Data folder)
birch = pd.read_csv("/..location../Data/birch.csv")
black_spruce = pd.read_csv("/..location../Data/black_spruce.csv")
trail = pd.read_csv("/..location../Data/trail.csv")

In [ ]:
birch = birch["Mean_birch_Reflectance"]
black_spruce = black_spruce["Mean_black_spruce_Reflectance"]
trail = trail["Mean_trail_Reflectance"]

print (trail.shape)

In [ ]:
srf_mss = pd.read_csv("/..location../Data/srf_mss.csv") #path to the spectral response function (SRF) of Sentinel data

srf_mss=srf_mss.drop(srf_mss.columns[0:2], axis=1)
srf_mss_transpose=srf_mss.T
print (srf_mss.shape)
print (srf_mss_transpose.shape)
#print (srf_mss)

In [ ]:
#reshape standard spectra
birch = birch.values.reshape(2150,1)
black_spruce = black_spruce.values.reshape(2150,1)
trail = trail.values.reshape(2150,1)

In [ ]:
#matrix multiplication 
R_birch=np.dot(srf_mss_transpose, birch)
R_black_spruce=np.dot(srf_mss_transpose, black_spruce)
R_trail=np.dot(srf_mss_transpose, trail)

In [ ]:
RM = np.concatenate((R_birch, R_black_spruce, R_trail), axis=1)
print (RM.shape)
print (RM)
RM_t=RM.T

In [ ]:
#part second hyperspectral simulation
srf_hss = pd.read_csv("/..location../Data/AVIRIS_SRF.csv",sep=',') #path to the spectral response function (SRF) of AVIRIS-NG data
srf_hss=srf_hss.drop(srf_hss.columns[0:2], axis=1)
srf_hss_transpose=srf_hss.T
print (srf_hss.shape)
print (srf_hss_transpose.shape)

In [ ]:
#matrix multiplication 
Rh_birch=np.dot(srf_hss_transpose, birch)
Rh_black_spruce=np.dot(srf_hss_transpose, black_spruce)
Rh_trail=np.dot(srf_hss_transpose, trail)

RH = np.concatenate((Rh_birch, Rh_black_spruce, Rh_trail), axis=1)

print (RM_t.shape)

In [ ]:
#CHANGE path: Path for Sentinel image
src_path = Path("/path/S2B_T06VXP.tif")  # Specify input image path"

In [ ]:
dst_tag = "HS"
dst_dir = Path("/path/output") #set output path
dst_dir.mkdir(mode=0o755, parents=True, exist_ok=True)
tiles_dir = dst_dir / "Tiles"
tiles_dir.mkdir(mode=0o755, parents=True, exist_ok=True)

#Set tile size
win_height = 2048  # Change as required
win_width = 2048  # Change as required

In [ ]:
with rio.open(src_path, 'r') as src:
    meta = src.profile.copy()
    img_h = src.height
    img_w = src.width
    big_win = Window(row_off=0, col_off=0, height=img_h, width=img_w)
    r_offsets = list(range(0, src.height, win_height))
    c_offsets = list(range(0, src.width, win_width))
    r_indexes = list(range(len(r_offsets)))
    c_indexes = list(range(len(c_offsets)))
    offsets = list(product(r_offsets, c_offsets))
    indexes = list(product(r_indexes, c_indexes))
    pointers = list(zip(indexes, offsets))
    
    # update meta for output image as required
    meta['count'] = RH.shape[0]
    meta['dtype'] = np.float32
    meta['nodata'] = np.nan
    meta['BIGTIFF'] = True
    tiles = list()

    for (i, j), (r_off, c_off) in tqdm(pointers):
      win = Window(
        row_off=r_off, col_off=c_off, height=win_height, width=win_width
      ).intersection(big_win)
      img = src.read(window=win, boundless=False, masked=True) # This is a 3D array (band x Rowsx Cols)
      
      # processing 
      dp = img.filled()
      dp = np.moveaxis(dp, -1, 0)
      dp = np.dot(RM_t, dp)
      dp = np.moveaxis(dp, -1, 0)
      dp = np.dot(np.linalg.inv(np.dot(RM_t,RM)), dp)
      dp = np.moveaxis(dp, -1, 0)
      dp = np.dot(RH, dp)
      dp = np.moveaxis(dp, -1, 1)

      out = dp.astype(meta['dtype'])
      mask = np.any(img.mask, axis=0, keepdims=True)
      mask = np.repeat(mask, meta['count'], axis=0)
      out[mask] = meta['nodata']

      # Write processed array to file
      dst_path = tiles_dir / "{}_{}_{}_{}{}".format(
        src_path.stem, dst_tag, i, j, src_path.suffix
      )
      meta['height'] = win.height
      meta['width'] = win.width
      meta['transform'] = src.window_transform(win)
      with rio.open(dst_path, 'w', **meta) as dst:
        dst.write(out) # Band id needs to be soecified in case of writing a 2D array 
      tiles.append(str(dst_path.relative_to(dst_dir)))
    os.chdir(dst_dir)
    vrt_path = dst_dir / '{}_{}.{}'.format(src_path.stem, dst_tag, "VRT")
    vrt_options = gdal.BuildVRTOptions(resampleAlg='near', addAlpha=False)
    ds = gdal.BuildVRT(
    str(vrt_path.relative_to(dst_dir)), tiles, options=vrt_options
    )
    ds.FlushCache()